# Overview
This notebook is intended to demonstrate how to run a differential gene expression analysis between tumor and normal samples using GDC cancer data made available in BigQuery. We will be using the TCGA gene expression dataset in `isb-cgc-bq`, specifically the table `isb-cgc-bq.TCGA.mRNAseq_hg38_gdc_current`. This notebook is modeled to somewhat mirror the differential expression analysis found in Peng, et al. *Scientific Reports* 2015 (https://doi.org/10.1038/srep13413).

First we need to set up the environment and authenticate our session. This notebook will use both python to construct and run our SQL query and the Bioconductor R libraries `edgeR` and `DESeq2`.

In [ ]:
library(DESeq2)
library(edgeR)
library(scales)
library(bigrquery)
project <- 'isb-cgc-outreach'
bigrquery::bq_auth(path = "~/key-file")

# Finding all cases with data for both normal and tumor aliquots
Our first step is to join sample_type data to our RNA seq table and identify the cases in the RNA seq BigQuery table of choice that have data for both tumor and normal aliquots. To perform this operation we can write an SQL query which groups rows by case barcode and then use the `array_agg()` function to concatenate the `sample_type_name` field for each case. 

In [ ]:
rna_table <- "isb-cgc-bq.TCGA_versioned.RNAseq_hg38_gdc_r35"

case_query <- "
WITH rna as (
    SELECT 
       case_barcode, 
       sample_barcode, 
       aliquot_barcode, 
       Ensembl_gene_id_v, 
       gene_name, 
       HTSeq__Counts,
       HTSeq__FPKM_UQ,
       sample_type_name,
    FROM `isb-cgc-bq.TCGA.RNAseq_hg38_gdc_current`
    WHERE gene_type = 'protein_coding'
    AND project_short_name = 'TCGA-BRCA'
),
cases as (
  SELECT case_barcode, agg FROM
      (SELECT 
        case_barcode, 
        array_agg(distinct sample_type_name) agg
      FROM rna
      GROUP BY case_barcode)
  WHERE array_length(agg) > 1
  AND ('Solid Tissue Normal' in UNNEST(agg))
  )"

select_cases <- "
SELECT * FROM cases
"

#print(paste0(case_query, select_cases))
case_job <- bq_project_query( project, query=paste0(case_query, select_cases) )
case_df <- as.data.frame(bq_table_download(case_job))
head(case_df)

# Subsetting the gene set
Performing a comparison across all genes is outside of the scope of this notebook, so we will reduce our gene set to a more manageable size. A common metric to subset by is the variance of expression level. We can calculate this metric in an SQL query by grouping by the `Ensembl_gene_id_v` field and calculating the `VARIANCE()` of the counts field. We then order our results by this value and select the top 2000 genes.

In [ ]:
expression_query <- ",
mean_expr as (
  SELECT * FROM (
    SELECT  
      rna.Ensembl_gene_id_v, 
      VARIANCE(rna.HTSeq__FPKM) var_fpkm
    FROM rna 
    JOIN cases ON rna.case_barcode = cases.case_barcode 
    WHERE rna.sample_type_name = 'Solid Tissue Normal'
    GROUP BY rna.Ensembl_gene_id_v)
  ORDER BY var_fpkm DESC
  LIMIT 2000)"

select_genes <- "
SELECT * FROM mean_expr
"

expr_job <- bq_project_query( project, query=paste0(case_query, expression_query, select_genes) )
# print( paste0( case_query + expression_query + select_genes ) )
expr_data <- as.data.frame( bq_table_download(expr_job) )
head(expr_data)

# Joins are used to generate the table for analysis
To generate the final table we join the full RNA expression table back to the tables housing the list of cases as well as the list of genes. As both `edgeR` and `DESeq2` require counts as input the important fields we need are `case_barcode`, `aliquot_barcode`, `gene_name`, `sample_type_name`, and `HTSeq__Counts`.

In [ ]:
final_join = "
SELECT rna.case_barcode, 
       rna.sample_barcode, 
       rna.aliquot_barcode, 
       rna.Ensembl_gene_id_v, 
       rna.gene_name, 
       rna.HTSeq__Counts,
       rna.sample_type_name,
FROM rna
  JOIN cases ON rna.case_barcode = cases.case_barcode
  JOIN mean_expr ON rna.Ensembl_gene_id_v = mean_expr.Ensembl_gene_id_v
WHERE rna.sample_type_name = 'Solid Tissue Normal' 
  OR rna.sample_type_name = 'Primary Tumor'
ORDER BY Ensembl_gene_id_v, case_barcode, aliquot_barcode 
"

final_job <- bq_project_query( project, paste0(case_query, expression_query, final_join) )
# print( case_query + expression_query + final_join )
final_data <- as.data.frame( bq_table_download(final_job) )
head( final_data )

# Converting our longform data table into a count matrix
Both `edgeR` and `DESeq2` expect a data matrix of counts as an input where each column corresponds to a sample and each row to a gene. In our query we sorted our data frame by the `gene_name`, `case_barcode`, and `aliquot_barcode`. This allows us to split our data frame on gene_name and then `rbind()` the resulting count vectors into a matrix.

In [ ]:
split_df <- split(final_data, f=final_data$gene_name)
str(split_df[c(1,2)]) # showing two data frames from our list output
expression_list <- lapply(split_df, function(df){df$HTSeq__Counts})
exp_matrix <- as.data.frame(Reduce(rbind, expression_list))

We load our genes as rownames and the aliquot_barcodes as column names

In [ ]:
colnames(exp_matrix) <- split_df[[1]]$aliquot_barcode
genes <- names(split_df)
rownames(exp_matrix) <- genes
str(exp_matrix, list.len=5)

We also need to generate vectors of which case each aliquot corresponds to as well as whether the aliquot is a tumor or normal sample. We can easily retrieve this information from our split data frame.

As a quick sanity check we can generate a heatmap to look at how our aliquots cluster and generate a histogram to compare the distributions of normal and tumor expression values.

In [ ]:
colnames(exp_matrix) <- split_df[[1]]$id
cases <- gsub('-', '.', split_df[[1]]$case_barcode)
str(cases)
sample_type <- gsub(' ', '.', split_df[[1]]$sample_type_name)
str(sample_type)
heatmap(as.matrix(exp_matrix), labCol=sample_type, scale="row")

In [ ]:
norm <- log10(unlist(exp_matrix[,which(sample_type == 'Solid.Tissue.Normal')]))
tumor <- log10(unlist(exp_matrix[,which(sample_type == 'Primary.Tumor')]))

p_norm <- hist(norm, breaks=seq(0,8,0.1), plot=0)
p_tumor <- hist(tumor, breaks=seq(0,8,0.1), plot=0)
plot( p_norm, col=rgb(0,0,1,1/8), xlim=c(0,8), xlab='log10(count)', main='')
plot( p_tumor, col=rgb(1,0,0,1/8), xlim=c(0,8), add=T)
legend("topright", legend=c("Normal", "Tumor"), fill=c(rgb(0,0,1,1/8), rgb(1,0,0,1/8)), bty="n") 

# Calculating differential expression using edgeR
We can now feed these data to edgeR, generate significance values of differential expression, and visualize the results using a scatterplot. Genes that fall below a 0.01 significance threshold are color coded red.

In [ ]:
dge_obj <- DGEList(
             counts=exp_matrix, 
             lib.size=colSums(exp_matrix),
             samples=cases, 
             group=sample_type)
disp <- estimateDisp(dge_obj, method="pooled")
edgeR_test <- exactTest(disp)
results_df <- topTags(edgeR_test, nrow(exp_matrix))@.Data[[1]]
str(results_df)

In [ ]:
colors <- c(alpha('red', 0.3), alpha('black', 0.6))[(results_df$FDR > 0.01)+1]
plot(results_df$logCPM, results_df$logFC, 
     pch=19, cex=1.7, col=colors,
     xlab='log(fold change)', ylab='log(counts per million)')
sum(results_df$FDR <= 0.01)

In [ ]:
col_info <- as.data.frame(cbind(cases, sample_type), stringAsFactors=TRUE)
deseq_obj <- DESeqDataSetFromMatrix( countData = exp_matrix, colData= col_info, design= ~ cases + sample_type)
deseq_output <- DESeq(deseq_obj, fitType='local')
summary(deseq_output)
deseq_results <- results(deseq_output)@listData

In [ ]:
colors <- c(alpha('red', 0.3), alpha('black', 0.6))[(deseq_results$padj > 0.01)+1]
plot(log(deseq_results$baseMean), deseq_results$log2FoldChange, 
     pch=19, cex=1.7, col=colors,
     xlab='log(mean expression)', ylab='log2(fold change)')
sum(deseq_results$padj <= 0.01)

# Summary
It is quite simple to retrieve expression data from the GDC through the BQ ecosystem. These data can be subset and summarized easily via SQL queries and the results loaded into either R or Python for further analysis and visualization.